In [6]:
from jupyterImport import enableJupyterImports
enableJupyterImports()
from siblings import offeringsWithProfessors
import json

In [7]:
from copy import deepcopy
limit = 10*1024
overLimit = lambda off: calcSize(off) > limit
calcSize = lambda dct: len(json.dumps(dct))

def strippedResponse(response):
  stripped = {'score': response['score']}
  if 'percentiles' in response and 'similar' in response['percentiles']:
    stripped['percentiles'] = {'similar': response['percentiles']['similar']}
  return stripped

def stripReport(reportOrProf):
  if 'responses' in reportOrProf:
    for attribute, response in reportOrProf['responses'].iteritems():
      reportOrProf['responses'][attribute] = strippedResponse(response)
  return reportOrProf

def trimmedReports(reports, targetSize):
  trimmed = deepcopy(reports)
  for i, report in enumerate(reversed(trimmed)):
    stripReport(report)
    if 'profs' in report:
      report['profs'] = [stripReport(p) for p in report['profs']]        
    if calcSize(trimmed) < targetSize:
#       print '  Got under size limit by stripping', i + 1, 'of', len(reports)
      return trimmed 
  while calcSize(trimmed) > targetSize:
#     print '  Still too big', calcSize(trimmed)
    if len(trimmed) > 1:
      trimmed = trimmed[:-1]
    else:
#       print '  Removed all', len(reports), 'to get under size limit.'
      return []
#   print '  Removed', len(reports), '->', len(trimmed), 'to get under size limit.',
#   print '  Last:', reports[-1]['term'], reports[-1]['year'], '->', trimmed[-1]['term'], trimmed[-1]['year']
  return trimmed

def trimProfs(newOffering):
  for prof in newOffering['profs']:
    if 'matched' in prof:
      del prof['matched']

def trimTopReport(newOffering):
  if 'topReport' in newOffering:
    top = newOffering['topReport']
    del newOffering['topReport']
    trimmedTop = trimmedReports([top], 10*1024 - calcSize(newOffering))
    if len(trimmedTop) > 0:
      newOffering['topReport'] = trimmedTop[0]
      
def trimReports(newOffering):
  oldReports = [r for r in newOffering['reports'] if r['year'] < '2014']
  newReports = [r for r in newOffering['reports'] if r['year'] >= '2014']
  del newOffering['reports']
  if overLimit(newOffering):
    trimTopReport(newOffering)
  if (calcSize(newOffering) + calcSize(newReports)) < limit:
    newOffering['reports'] = newReports + trimmedReports(oldReports, limit - (calcSize(newOffering) + calcSize(newReports))) 
  elif calcSize(newOffering) < limit:
    trimmedNew = trimmedReports(newReports, limit - calcSize(newOffering))
    if len(trimmedNew) > 0:
      newOffering['reports'] = trimmedNew

from collections import Counter
def algoliaCompress(offerings):
  algoliaOfferings = []
  for offering in offerings:
    if overLimit(offering):
      newOffering = deepcopy(offering)
      if 'reports' in newOffering:
        trimReports(newOffering)
      if overLimit(newOffering):
        trimTopReport(newOffering)
      if overLimit(newOffering):
        trimProfs(newOffering)
      if overLimit(newOffering):
        raise
      else:
        print offering['group'], offering['number']
        print '  from', calcSize(offering), '->', calcSize(newOffering)
        algoliaOfferings.append(newOffering)
    else:
      algoliaOfferings.append(offering)
  return algoliaOfferings

In [8]:
algoliaOfferings = algoliaCompress(offeringsWithProfessors)

MUSIC 207R
  from 10568 -> 9821
FRENCH 30
  from 23058 -> 9667
FRENCH 30
  from 23058 -> 9667
WOMGEN 99B
  from 14144 -> 10175
ETHRSON 33
  from 14370 -> 9161
PHIL 300B
  from 11865 -> 10187
COMPSCI 179
  from 15732 -> 9465
VIROLOGY 202
  from 34912 -> 8277
CHNSHIS 113
  from 12120 -> 8416
PHYSICS 253B
  from 13756 -> 9684
FRSEMR 41K
  from 10983 -> 9117
PSY 971
  from 20786 -> 9733
PSY 971
  from 20788 -> 9735
RELIGION 3002
  from 15676 -> 8805
KOREAN BB
  from 22539 -> 9711
ECON 2490
  from 15660 -> 9504
SOC-STD 98LF
  from 15162 -> 9918
SLAVIC 97
  from 24257 -> 9942
OEB 50
  from 16715 -> 8983
MED-SCI 302QC
  from 10753 -> 9947
KOREAN 150B
  from 13194 -> 8759
AFRAMER 16
  from 10316 -> 8816
CHEM 60
  from 16079 -> 9754
LATIN 112B
  from 16926 -> 10182
ENGLISH CBBR
  from 15333 -> 9823
ECON 1420
  from 27401 -> 9211
FRSEMR 35N
  from 20410 -> 10121
PHYSICS 210
  from 15903 -> 10096
AFRAMER 11
  from 29425 -> 9360
GOV 2755
  from 11367 -> 10107
SOC-STD 10B
  from 22057 -> 10034
HIND

In [10]:
len(algoliaOfferings)

13455

In [11]:
from algoliasearch import algoliasearch
client = algoliasearch.Client("D86FK05FUD", '0cebfab0abe857cd36dd70e37f043a64')
index = client.init_index("offerings")
index.save_objects(algoliaOfferings)

{u'objectIDs': [u'113307_15826',
  u'113307_15827',
  u'113307_15824',
  u'113307_15825',
  u'116391_12689',
  u'113704_0',
  u'104604_19711',
  u'113307_15828',
  u'170226_0',
  u'117788_15901',
  u'110819_0',
  u'160536_13905',
  u'108658_18545',
  u'116549_15165',
  u'116549_15166',
  u'191143_11769',
  u'124526_0',
  u'122706_14509',
  u'122706_14508',
  u'122706_14507',
  u'110638_19745',
  u'191143_11767',
  u'122706_14504',
  u'122706_14503',
  u'122592_18337',
  u'108854_16480',
  u'205052_18436',
  u'111282_11990',
  u'115669_12733',
  u'112844_12107',
  u'116549_15162',
  u'112838_15296',
  u'116549_15163',
  u'205477_10218',
  u'126942_13661',
  u'126942_13660',
  u'190419_10591',
  u'121266_13092',
  u'119079_14209',
  u'119079_14208',
  u'119079_14207',
  u'113307_15829',
  u'180849_18888',
  u'112838_15292',
  u'116418_12710',
  u'112981_16569',
  u'111378_0',
  u'114925_12456',
  u'170499_18451',
  u'112981_16564',
  u'112981_16567',
  u'112838_15290',
  u'190222_11494',